# Query MR-EvE to identify potential mediators

We can query MR-EvE relationships to extract potential mediators between an exposure and an outcome. The basic principle is similar to the example in `01_mr-eve_exposure_to_outcomes`. We run a query that essentially finds all two-step MR results, where step 1 outcomes intersect with step 2 exposures. The intersection traits are the potential mediators. This query has to be run with some p-value threshold to limit the output to plausible results.


This type of query can be used to find not only mediators but also confounders, colliders, and reverse mediators. This can be achieved by tweaking the exposure/outcome relationships between the three traits in the query. See the EpiGraphDB page ([https://epigraphdb.org/confounder](https://epigraphdb.org/confounder)) dedicated to building these queries (use _Query_ tab to generate the appropriate query).


### Example for querying mediaotrs

In this example we assume that the evidence of effect in `exposure -> outcome` relationship has been validated (otherwise looking for mediators does not make sense).

In [8]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(epigraphdb))

In [5]:
#  function that will export query result as a table
query_epigraphdb_as_table <- function(query){
  results_subset <- query_epigraphdb(
    route = "/cypher",
    params = list(query = query),
    method = "POST",
    mode = "table")
}

In [13]:
# specify exposure and outcome ids and p-value threshold for two-step MR steps
exposure_id = "prot-a-670"
outcome_id = "ieu-a-1127"

pval_threshold_step1 = 0.01 # exposure -> mediator
pval_threshold_step2 = 0.01 # mediator -> outcome

In [14]:
mediator_query = paste0("
    MATCH (med:Gwas)<-[r1:MR_EVE_MR]- (exposure:Gwas) -[r2:MR_EVE_MR]->(outcome:Gwas) <-[r3:MR_EVE_MR]-(med:Gwas) 
    WHERE exposure.id = '", exposure_id, "' 
    AND outcome.id = '", outcome_id,"' 
    AND (not (toLower(med.trait) contains 'breast')) 
    AND r1.pval < ", pval_threshold_step1, " AND r3.pval < ", pval_threshold_step2, " 
    AND med.id <> exposure.id 
    AND med.id <> outcome.id 
    AND exposure.id <> outcome.id 
    AND med.trait <> exposure.trait 
    AND med.trait <> outcome.trait 
    AND exposure.trait <> outcome.trait 
    RETURN  exposure {.id, .trait}, 
            outcome {.id, .trait},
            med {.id, .trait}, 
            r1 {.b, .se, .pval, .method, .moescore},
            r2 {.b, .se, .pval, .method, .moescore}, 
            r3 {.b, .se, .pval, .method, .moescore} ORDER BY r1.p
    ")

In [15]:
mediators = query_epigraphdb_as_table(mediator_query)

In [17]:
head(mediators)

exposure.trait,exposure.id,outcome.trait,outcome.id,med.trait,med.id,r1.b,r1.se,r1.method,r1.pval,⋯,r2.se,r2.b,r2.method,r2.pval,r2.moescore,r3.se,r3.b,r3.method,r3.pval,r3.moescore
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
C-reactive protein,prot-a-670,ER+ Breast cancer (Combined Oncoarray; iCOGS; GWAS meta analysis),ieu-a-1127,HDL cholesterol,ukb-d-30760_raw,0.012319200,0.002780567,FE IVW,9.403107e-06,⋯,0.02417862,0.06824084,FE IVW,0.004767127,1,0.08694217,0.2376427,Penalised median,6.269509e-03,0.74
C-reactive protein,prot-a-670,ER+ Breast cancer (Combined Oncoarray; iCOGS; GWAS meta analysis),ieu-a-1127,HDL cholesterol,ukb-d-30760_irnt,0.037280011,0.007357905,FE IVW,4.048547e-07,⋯,0.02417862,0.06824084,FE IVW,0.004767127,1,0.02447639,0.1000475,Simple mean,5.935667e-05,0.70
C-reactive protein,prot-a-670,ER+ Breast cancer (Combined Oncoarray; iCOGS; GWAS meta analysis),ieu-a-1127,Impedance of arm (right),ukb-b-7859,0.018535949,0.005073919,FE IVW,2.590103e-04,⋯,0.02417862,0.06824084,FE IVW,0.004767127,1,0.03194656,0.1953487,FE IVW,9.664238e-10,0.82
C-reactive protein,prot-a-670,ER+ Breast cancer (Combined Oncoarray; iCOGS; GWAS meta analysis),ieu-a-1127,Age last used hormone-replacement therapy (HRT),ukb-b-4564,0.035896891,0.013838204,FE IVW,9.485472e-03,⋯,0.02417862,0.06824084,FE IVW,0.004767127,1,0.11975685,0.3710881,FE IVW,1.943854e-03,1.00
C-reactive protein,prot-a-670,ER+ Breast cancer (Combined Oncoarray; iCOGS; GWAS meta analysis),ieu-a-1127,Illnesses of father: Alzheimer's disease/dementia,ukb-b-323,0.004127199,0.001231144,FE IVW,8.013470e-04,⋯,0.02417862,0.06824084,FE IVW,0.004767127,1,0.14189943,-0.9494398,FE IVW,2.217495e-11,1.00
C-reactive protein,prot-a-670,ER+ Breast cancer (Combined Oncoarray; iCOGS; GWAS meta analysis),ieu-a-1127,"Never eat eggs, dairy, wheat, sugar: Wheat products",ukb-b-3599,-0.001507927,0.000163362,FE IVW,2.691546e-20,⋯,0.02417862,0.06824084,FE IVW,0.004767127,1,0.19270422,-0.9401876,FE IVW,1.066709e-06,1.00


In the results table,

```
r1 is exposure -> mediator
r2 is exposure -> outcome
r3 is mediator -> outcome
```